### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [1]:
import numpy as np
import data_gen as dg
data1=dg.data_gen()
y=data1.y
x=data1.x
e=data1.e
print(type(x))
n,p=x.shape

<class 'numpy.ndarray'>


### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
 - 'product'-Ndiff1 multiplied by Ndiff2
 - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
 - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
 - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- kern_grid: 
 - no means smooth the original data
 - int means form a grid across all the variables for xin, yxin with int evenly spaced 
 observations (could make flexible than uniform distribution)
- Ndiff_bw_kern:
 - rbfkern means use the radial basis function kernel
 - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
 - 'NW' means use nadaraya-watson kernel regression
 - 'full_logit' means local logit with all variables entering linearly
 - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- kern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data, which is useful for calibrating hyper parameters
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random 
variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels 
  across that random variable (as if creating a cdf)
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - p_bandwidth is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [2]:
modeldict1={
    'Ndiff_type':'product',
    'max_bw_Ndiff':2,
    'normalize_Ndiffwtsum':'own_n',
    'kern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'none',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'p_bandwidth':'fixed',
        'Ndiff_depth_bw':'free',
        'outer_x_bw':'free',
        'outer_y_bw':'free'
        }
    }

### set starting or fixed hyper parameter values

In [3]:
hyper_paramdict1={
    'Ndiff_exponent':np.zeros([2,]),
    'p_bandwidth':np.ones([p,]),
    'outer_x_bw':np.array([0.3,]),
    'outer_y_bw':np.array([0.3,]),
    'Ndiff_depth_bw':np.zeros([2,])
                  }
assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
                                                                             'depth of max_bw_Ndiff'
assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
                                                                             'depth of max_bw_Ndiff'

### create optimization dictionary

In [4]:
optimizedict1={
    'method':'Nelder-Mead',
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

#-----------------Calibrate/Optimize--------
#-------find values for hyperparameters-----

In [5]:
import mykern as mk

#kerneltool1=mk.kNdtool()
#kerneltool1.optimize_free_params(y,x,optimizedict1)

#inheritance version
#optimized_Ndiff_kernel=mk.kNdtool.optimize_free_params(y,x,optimizedict1)
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)


print(optimized_Ndiff_kernel.mse)
import myreg
just_linreg=myreg.reg(y,x[:,1:])#drop first column of x (all 1's)
print(just_linreg.mse)

(200,)
(200, 3)


C:\Users\DPatton\PycharmProjects\kernelkernel\mykern.py:396: RuntimeWarning: invalid value encountered in true_divide
  standard_x=(xdata-self.xmean)/self.xstd


NameError: name 'nout' is not defined